In [ ]:
%%javascript
$('#header').toggle();

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, Heatmap, Circle, Polyline
from ipywidgets import HTML, Label, Valid, Combobox, Dropdown, Text, Button, VBox, HBox, Box
from ipywidgets import Layout, AppLayout, GridspecLayout
from geopy.geocoders import Nominatim, BANFrance
from geopy.distance import geodesic
import numpy as np
import pandas as pd
from datetime import datetime


maps = {'Standard' : basemaps.OpenStreetMap.Mapnik}
basemap_selector_value = 'Standard'

countries_file = 'sql-pays.csv'
state = sorted(list(pd.read_csv(countries_file, sep=',', low_memory=False, header=None)[4]))

geolocator = Nominatim(user_agent="appMAP_cernicchiaro.giulia@gmail.com")
geolocator_fr = BANFrance()


glb = {'curr_locs' : {},
       'pane_heights' : ['12%', '35%', '48%'],
       'center_location' : None,
       'log_rows' : []
      }


header = HTML("<h2>GPS à Vol d'Oiseaux</h2>", layout=Layout(height='auto'))
header.style.text_align='center'

state_selector = Dropdown( options = list(state), layout=Layout(width='auto'), value = 'France' )

subheader_1 = HTML(
    value='<h3>Adresse choisie</h3>',
    layout=Layout(width='auto', height='auto')
)

user_search = Text(placeholder='Saisir une adresse et cherhcer OU cliquer sur la carte')
user_search.layout.width = '90%'

user_action = Button(description='Chercher', button_style='info', tooltip="Geolocaliser l'adresse saisie")

subheader_2 = HTML(
    value='<h4>Adresses Trouvées</h4>',
    layout=Layout(width='auto', height='auto')
)

address_selector = Dropdown()
address_selector.layout.width = '98%'

main_map = Map(basemap=basemaps.OpenStreetMap.Mapnik)
main_map.layout.height = 'auto'
center_map = geolocator.geocode(state_selector.value)
main_map.center = (center_map.latitude, center_map.longitude)
main_map.zoom = 6

circle = Circle()
circle_radius = Circle()

subheader_3 = HTML(
    value='<h3>Verifier une Adresse</h3>',
    layout=Layout(width='auto', height='auto')
)

user_verify = Text(placeholder="Saisir une adresse à verifier à vol d'oiseau")
user_verify.layout.width = '90%'

user_action_2 = Button(description='Verifier', button_style='info', tooltip="")

out = HTML(
    value='',
    layout=Layout(width='auto', height='auto')
)


grid_header = GridspecLayout(1, 4, height='auto')
grid_header[0, :3] = header
grid_header[0, 3] = VBox([Label("Pays : "),state_selector])

vbox_header = VBox([grid_header,
                    subheader_1,
                    HBox([user_search,user_action]),
                   ])

vbox_footer = VBox([out])


def add_log(msg):
    max_rows = 15
    glb['log_rows'].append(msg)
    if len(glb['log_rows']) > max_rows:
        glb['log_rows'].pop(0)
    out_rows=glb['log_rows'][:]
    out_rows.reverse()
    return '<h3>Activity log</h3><ul>{}</ul>'.format('<li>'.join([''] + out_rows))

def search_address(locs) :
    glb['curr_locs'] = {}
    msg_add = ' -> '
    if locs :
        for l in locs : 
            glb['curr_locs'][l.address] = l
        address_selector.options = list(['']+list(glb['curr_locs'].keys()))
        circle.location = (locs[0].latitude, locs[0].longitude)
        circle.radius = 1
        circle.color = "blue"
        main_map.layers = [basemap_to_tiles(maps[basemap_selector_value]), circle]
        circle_radius.location = (0, 0)
        circle_radius.radius = 0
        if len(locs) == 1 :
            msg_add += 'Adresse unique trouvée : ' + str(locs[0].address)
            state_selector.value = locs[0].raw['address']['country']
            vbox_header.children = tuple(list(vbox_header.children)[:3])
            user_search.placeholder = 'Saisir une adresse et cherhcer OU cliquer sur la carte'
            user_search.value = locs[0].address
        elif len(locs) > 1 :
            msg_add += str(len(locs)) + ' adresses trouvées'
            if state_selector.value not in [l.raw['address']['country'] for l in locs]:
                if len(set([l.raw['address']['country'] for l in locs])) == 1 :
                    state_selector.value = list(set([l.raw['address']['country'] for l in locs]))[0]
                else:
                    main_map.zoom = 3
            vbox_header.children = tuple(list(vbox_header.children)[:3] + [subheader_2, Box([address_selector])])
            user_search.placeholder = 'Choisir une adresse parmi celles trouvées OU Chercher une autre adresse'
            user_search.value = ''
            for l in locs[1:] :
                main_map.add_layer(Circle(location=(l.latitude, l.longitude),radius=1,color="blue"))
        user_action.description = 'Valider'
        vbox_footer.children = tuple([list(vbox_footer.children)[-1]])
    else :
        msg_add += '[ERROR] Aucune adresse trouvée'
        address_selector.options = list([''])
        vbox_header.children = tuple(list(vbox_header.children)[:3])
        user_search.placeholder = 'Aucun resultat pour ' + user_search.value + '. Ressayer'
        user_search.value = ''
    return msg_add


def on_state_selected(change):
    if (geolocator.geocode(state_selector.value, language='fr')):
        center_map = geolocator.geocode(state_selector.value)
        main_map.center = (center_map.latitude, center_map.longitude)
        main_map.zoom = 6
        msg = '( %s ) Pays selectioné : %s \n' % (datetime.now(), state_selector.value)    
    else:
        msg = '( %s [ERROR] ) %s : pays non disponible \n' % (datetime.now(), state_selector.value)    
    out.value = add_log(msg)

def on_text_entry(change):
    if user_search.value != '' :
        if user_search.value not in address_selector.options :
            user_action.description = 'Chercher'
            msg = '( %s ) Nouvelle recherche \n' % (datetime.now())    
        else :
            user_action.description = 'Valider'
            msg = '( %s ) Recherche à valider \n' % (datetime.now())    
        #out.value = add_log(msg)

def on_button_clicked(b):
    if user_search.value != '' :
        if user_action.description == 'Chercher':
            msg = '( %s ) Adresse saisie : %s \n' % (datetime.now(), user_search.value)    
            msg += search_address(geolocator.geocode(user_search.value,exactly_one=False,limit=1000000000000,addressdetails=True,language='fr'))
        elif user_action.description == 'Valider':
            msg = '( %s ) Adresse validée : %s \n' % (datetime.now(), user_search.value)    
            found_address = glb['curr_locs'][user_search.value]
            glb['curr_locs'] = {}
            address_selector.options = list([''])
            circle.location = (found_address.latitude, found_address.longitude)
            circle.radius = 1
            circle.color = "blue"
            main_map.layers = [basemap_to_tiles(maps[basemap_selector_value]), circle]
            circle_radius.location = (found_address.latitude, found_address.longitude)
            circle_radius.radius = 100000
            circle_radius.color = "red"
            circle_radius.fill_color = "green"
            main_map.add_layer(circle_radius)
            center_map = found_address
            main_map.center = (center_map.latitude, center_map.longitude)
            main_map.zoom = 8
            vbox_header.children = tuple(list(vbox_header.children)[:3] + [HBox([Valid(value=True),Label(found_address.address+' : SELECTIONNEE. Descendre sous la carte pour voir les destinations.')])])
            user_search.placeholder = 'Saisir une adresse et cherhcer OU cliquer sur la carte'
            user_search.value = ''
            user_action.description = 'Chercher'
            vbox_footer.children = tuple([subheader_3,
                                          HBox([Valid(value=True,description='Adresse'),Label(found_address.address)]),
                                          HBox([user_verify,user_action_2]),
                                          out])
            glb['center_location'] = found_address
        out.value = add_log(msg)

def handle_interaction(**kwargs):
    if kwargs['type'] == 'click':
        if user_action.description == 'Valider' and str(kwargs['coordinates'][0])+', '+str(kwargs['coordinates'][1]) in [str(glb['curr_locs'][k].latitude)+', '+str(glb['curr_locs'][k].longitude) for k in list(glb['curr_locs'].keys())] :
            zoom_loc = 18
            zoom_loc_str = 'adresse'
        else :
            if main_map.zoom < 4:
                zoom_loc = 4
                zoom_loc_str = 'pays'
            elif main_map.zoom < 6:
                zoom_loc = 7
                zoom_loc_str = 'region'
            elif main_map.zoom < 8:
                zoom_loc = 9
                zoom_loc_str = 'departement'
            elif main_map.zoom < 12:
                zoom_loc = 15
                zoom_loc_str = 'commune'
            else :
                zoom_loc = 18
                zoom_loc_str = 'adresse'
        msg = '( %s ) Coordonées choisies : %s ( zoom map : %s - zoom crd : %s )\n' % (datetime.now(), str(kwargs['coordinates'][0])+', '+str(kwargs['coordinates'][1]), str(round(main_map.zoom)), zoom_loc_str)    
        msg += search_address(geolocator.reverse(str(kwargs['coordinates'][0])+', '+str(kwargs['coordinates'][1]),exactly_one=False,addressdetails=True,zoom=zoom_loc,language='fr'))
        out.value = add_log(msg)

def on_address_selected(change):
    user_search.value = address_selector.value
    if user_search.value == '' :
        msg = '( %s ) Adresse vide \n' % (datetime.now()) 
    else :
        msg = '( %s ) Adresse selectionnée : %s \n' % (datetime.now(), address_selector.value) 
    #out.value = add_log(msg)

def on_action_change(change):
    if user_action.description == 'Chercher':
        user_action.tooltip = "Geolocaliser l'adresse saisie"
    else:
        user_action.tooltip = "Selectionner l'adresse et construir le rayon"
    msg = '( %s ) Tooltip actualisé\n' % (datetime.now())    
    #out.value = add_log(msg)

def on_verify_clicked(b) :
    locs = geolocator.geocode(user_verify.value,exactly_one=False,limit=1000000000000,addressdetails=True,language='fr') #,country_codes=glb['center_location'].raw['address']['country_code'])
    msg = '( %s ) Adresse à verififer : %s \n' % (datetime.now(), user_verify.value)    
    if locs :
        user_verify.value = locs[0].address
        if len(locs) == 1 :
            msg += ' -> trouvée unique'
            vbox_footer.children = tuple(list(vbox_footer.children)[:3] + [list(vbox_footer.children)[-1]])
        elif len(locs) > 1 :
            msg += ' -> ' + str(len(locs)) + ' trouvées, premiere'
            vbox_footer.children = tuple(list(vbox_footer.children)[:3] + [Label(str(len(locs)) + ' adresses trouvées avec cette recherche, utiliser le panneau de recherche en haut pour explorer les resultats')] + [list(vbox_footer.children)[-1]])
        current_distance = geodesic((locs[0].latitude, locs[0].longitude),(glb['center_location'].latitude, glb['center_location'].longitude)).km
        if current_distance <= 100 :
            msg += ' dans le rayon'
            color_result = 'green'
            vbox_footer.children = tuple(list(vbox_footer.children)[:-1] + [HBox([Valid(value=True),Label('Accessible : ' + str(current_distance) + " km à vol d'oiseau")])] + [list(vbox_footer.children)[-1]])
        else:
            msg += ' hors rayon'
            color_result = 'red'
            vbox_footer.children = tuple(list(vbox_footer.children)[:-1] + [HBox([Valid(value=False),Label('Inaccessible : ' + str(current_distance) + " km à vol d'oiseau")])] + [list(vbox_footer.children)[-1]])
        main_map.layers = [basemap_to_tiles(maps[basemap_selector_value]), circle, circle_radius]
        main_map.add_layer(Circle(location=(locs[0].latitude, locs[0].longitude),radius=1,color=color_result))
        main_map.add_layer(Polyline(locations=[[locs[0].latitude, locs[0].longitude],[glb['center_location'].latitude, glb['center_location'].longitude]],fill=False,color=color_result))
    else :
        msg += ' -> non trouvée'
        user_verify.placeholder = 'Aucun resultat pour ' + user_verify.value + '. Ressayer'
        user_verify.value = ''
        vbox_footer.children = tuple(list(vbox_footer.children)[:3] + [list(vbox_footer.children)[-1]])
    out.value = add_log(msg)
    


state_selector.observe(on_state_selected, names='value')
user_search.observe(on_text_entry,names='value')
user_action.on_click(on_button_clicked)
user_action.observe(on_action_change,names='description')
address_selector.observe(on_address_selected,names='value')
main_map.on_interaction(handle_interaction)
user_action_2.on_click(on_verify_clicked)


AppLayout(header=vbox_header,
          center=main_map,
          footer=vbox_footer,
          pane_heights=glb['pane_heights'],
          height='2000px',
          grid_gap="1px")